In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [70]:
path='C:/Users/김경한/word2vec자료/'
file_name='Reviews.csv'

In [71]:
data = pd.read_csv(path+file_name,nrows=10000)
print(len(data))

10000


In [72]:
data=data[['Text','Summary']]
data.head(4)

,Text,Summary
0,I have bought several of the Vitality canned d...,Good Quality Dog Food
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised
2,This is a confection that has been around a fe...,"""Delight"" says it all"
3,If you are looking for the secret ingredient i...,Cough Medicine


In [73]:
data['Text']

0       I have bought several of the Vitality canned d...
1       Product arrived labeled as Jumbo Salted Peanut...
2       This is a confection that has been around a fe...
3       If you are looking for the secret ingredient i...
4       Great taffy at a great price.  There was a wid...
                              ...                        
9995    we switched from the advance similac to the or...
9996    Like the bad reviews say, the organic formula ...
9997    I wanted to solely breastfeed but was unable t...
9998    i love the fact that i can get this delieved t...
9999    We have a 7 week old... He had gas and constip...
Name: Text, Length: 10000, dtype: object

In [74]:
data['Text'].nunique()

9513

In [75]:
data.drop_duplicates(subset=['Text'],
                    inplace=True)

In [76]:
len(data)

9513

In [77]:
data.dropna(axis=0, inplace=True)

In [78]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"}

In [79]:
stopWords = set(stopwords.words("english"))
len(stopWords)

179

In [81]:
def preprocessSentence(sent, rs=True):
    sent = sent.lower()
    sent = BeautifulSoup(sent,"lxml").text #<- html 태그 제거
    sent = re.sub("\([^)]*\)", " ", sent) #<- 괄호로 묶여있는 부분 제거
    sent = re.sub('""', " ", sent) #<- 홑따옴표 안에 곁따옴표를 넣어야 인식함
    sent = " ".join([contractions[t] if
                     t in contractions else t for t in sent.split(" ")])
    sent = re.sub("'s\b"," ",sent) #<-소유격 제거
    sent = re.sub("[^a-zA-Z]"," ", sent) #<-영어외 제거
    sent = re.sub("[m]{2,}","mm", sent) #<- m이 2개이상이면 mm으로 바꾸기
    
    #rs==True 이면 불용어 제거(원문)
    if rs:
        tokens = " ".join(word for word in sent.split()
        if not word in stopWords 
        if len(word)>1)
    else: #<- 불용어 제거 안함(요약)
        tokens = " ".join(word for
                          word in sent.split() 
                          if len(word)>1)
    return tokens

In [82]:
preprocessSentence("<a>I ammmmmmm a student</a>")

'amm student'

In [83]:
cleanText=[]
for sent in data['Text']:
    cleanText.append(preprocessSentence(sent))
print(cleanText[:3])
data['Text']=cleanText

['bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better', 'product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo', 'confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story lewis lion witch wardrobe treat seduces edmund selling brother sisters witch']


In [85]:
cleanSummary=[]
for sent in data['Summary']:
    cleanSummary.append(preprocessSentence(sent))
print(cleanSummary[:2])

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'cookies'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


['good quality dog food', 'advertised']


In [86]:
data['Summary']=cleanSummary

In [87]:
data.replace("",np.nan,inplace=True)
data.isnull().sum()

Text        0
Summary    35
dtype: int64

In [88]:
data.dropna(axis=0, inplace=True)

In [89]:
textLen = [len(s.split()) for s in data['Text']]
summaryLen = [len(s.split()) for s in data['Summary']]

In [90]:
# 전처리 전 최소, 최대, 평균 길이를 다 확인해 보는 것이 좋음
np.min(textLen) #<- 2
np.max(textLen) #<- 1235
np.mean(textLen) #<- 38

36.63040725891538

In [91]:
np.min(summaryLen) #<- 1
np.max(summaryLen) #<- 16
np.mean(summaryLen) #<- 2.8

2.8470141380037983

In [92]:
textMaxLen=50
summaryMaxLen=8

In [93]:
def threshLen(mlen,nlist):
    c=0
    for s in nlist:
        if (len(s.split())<= mlen):
            c+=1
    print(c/len(nlist))

In [94]:
print(threshLen(textMaxLen, data['Text']))
threshLen(summaryMaxLen, data['Summary'])

0.7968980797636632
None
0.9935640430470564


In [95]:
data = data[data['Text'].apply(
    lambda x:len(x.split())<=textMaxLen)]

In [96]:
data = data[data['Summary'].apply(
    lambda x:len(x.split())<=summaryMaxLen)]

In [97]:
#seq2seq 
data['Summary'] = data['Summary'].apply(lambda x:
                                        "sostoken " + x + "eostoken")
data['Summary']

0       sostoken good quality dog foodeostoken
1                  sostoken advertisedeostoken
2                sostoken delight sayseostoken
3              sostoken cough medicineeostoken
4                 sostoken great taffyeostoken
                         ...                  
9993                     sostoken timeeostoken
9994            sostoken great producteostoken
9995             sostoken constipationeostoken
9996     sostoken constipation problemeostoken
9998               sostoken convenienteostoken
Name: Summary, Length: 7527, dtype: object

In [98]:
textData = list(data['Text'])
summaryData = list(data['Summary'])

In [99]:
from sklearn.model_selection import train_test_split

In [100]:
xTrain, xTest, yTrain, yTest = train_test_split(textData,
                 summaryData,
                 test_size = 0.2,
                random_state = 42,
                shuffle = True)

In [101]:
srcToken = Tokenizer()
srcToken.fit_on_texts(xTrain)
#단어 집합 생성

In [102]:
srcToken.word_index

{'like': 1,
 'good': 2,
 'great': 3,
 'coffee': 4,
 'taste': 5,
 'product': 6,
 'flavor': 7,
 'love': 8,
 'one': 9,
 'would': 10,
 'really': 11,
 'best': 12,
 'tea': 13,
 'buy': 14,
 'use': 15,
 'price': 16,
 'much': 17,
 'get': 18,
 'time': 19,
 'tried': 20,
 'amazon': 21,
 'little': 22,
 'make': 23,
 'find': 24,
 'food': 25,
 'chips': 26,
 'better': 27,
 'chocolate': 28,
 'cup': 29,
 'well': 30,
 'try': 31,
 'also': 32,
 'hot': 33,
 'bought': 34,
 'even': 35,
 'could': 36,
 'eat': 37,
 'found': 38,
 'mix': 39,
 'drink': 40,
 'favorite': 41,
 'delicious': 42,
 'sweet': 43,
 'recommend': 44,
 'made': 45,
 'order': 46,
 'free': 47,
 'sugar': 48,
 'used': 49,
 'first': 50,
 'tastes': 51,
 'cannot': 52,
 'easy': 53,
 'bag': 54,
 'flavors': 55,
 'dog': 56,
 'nice': 57,
 'loves': 58,
 'way': 59,
 'many': 60,
 'store': 61,
 'water': 62,
 'got': 63,
 'box': 64,
 'ordered': 65,
 'go': 66,
 'ever': 67,
 'milk': 68,
 'cups': 69,
 'still': 70,
 'perfect': 71,
 'since': 72,
 'without': 73,
 'makes

In [103]:
print(srcToken.num_index)

None


In [104]:
totalCnt = len(srcToken.word_index)

In [105]:
totalFreq = 0 #<- 전체 단어 빈도수 총 합
rCnt = 0 #<- 빈도수가 10 미만인 단어의 개수
rFreq = 0 #<- 빈도수가 10 미만인 단어 빈도수 총 합
for k,v in srcToken.word_counts.items():
    totalFreq+=v
    if (v<10):
        rCnt+=1
        rFreq+=v
print("단어 빈도수 총합:", totalFreq) 
print("단어수:", totalCnt)
print("빈도가 10 이하인 단어의 비율:",rCnt/totalCnt)
print("빈도수:", rFreq/totalFreq)
print("빈도수 10이상인 단어:", totalCnt-rCnt)


단어 빈도수 총합: 144320
단어수: 10939
빈도가 10 이하인 단어의 비율: 0.8216473169393912
빈도수: 0.14036169623059866
빈도수 10이상인 단어: 1951


In [106]:
srcVocab = 7000
srcToken = Tokenizer(num_words = srcVocab)
srcToken.fit_on_texts(xTrain)

In [107]:
xTrain = srcToken.texts_to_sequences(xTrain)
xTest = srcToken.texts_to_sequences(xTest)

In [108]:
print(xTrain[0])
data['Text'][0]

[115, 7, 15, 17, 480, 95, 210, 225, 45, 1186, 568, 30, 162, 5942, 30, 375, 161, 431, 376, 963, 119, 4467, 1096, 210, 4468, 1233, 15, 170, 1233, 1350, 1482, 7]


'bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better'

In [109]:
tarToken = Tokenizer()
tarToken.fit_on_texts(yTrain)

In [111]:
totalFreq = 0
rCnt = 0
rFreq = 0 

for k,v in tarToken.word_counts.items():
    totalFreq+=v
    if (v<10):
        rCnt+=1
        rFreq+=v
print("단어 빈도수 총합:", totalFreq) 
print("단어수:", totalCnt)
print("빈도가 10 이하인 단어의 비율:",rCnt/totalCnt)
print("빈도수:", rFreq/totalFreq)
print("빈도수 10이상인 단어:", totalCnt-rCnt)


단어 빈도수 총합: 21896
단어수: 10939
빈도가 10 이하인 단어의 비율: 0.33120029253131
빈도수: 0.3216569236390208
빈도수 10이상인 단어: 7316


In [112]:
tarVoc = 2000 
tarTokenizer = Tokenizer(num_words=tarVoc)
tarTokenizer.fit_on_texts(yTrain)


In [113]:
yTrain = tarTokenizer.texts_to_sequences(yTrain)
yTest = tarTokenizer.texts_to_sequences(yTest)

In [115]:
yTrain[:4]

[[1, 743, 127], [1, 1035, 103, 92], [1, 28, 88, 139], [1, 293]]

In [116]:
dropTrain = [i for i, sent in 
            enumerate(yTrain)
            if len(sent)==2]
dropTest = [i for i, sent in
           enumerate(yTest)
           if len(sent)==2]

In [117]:
xTrain = np.delete(xTrain, dropTrain, axis=0)
yTrain = np.delete(yTrain, dropTrain, axis=0)
xTest = np.delete(xTest, dropTest, axis=0)
yTest = np.delete(yTest, dropTest, axis=0)

In [119]:
len(xTrain)
len(xTest)

1072

In [120]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 128
hidden_size = 256

# encoder input layer
encoder_inputs = Input(shape = (textMaxLen,))

# encoder Embedding layer
enc_emb = Embedding(srcVocab, embedding_dim)(encoder_inputs)

# encoder lstm layer1
encoder_lstm1 = LSTM(hidden_size, return_sequences=True,
                    return_state=True,
                    dropout = 0.4,
                    recurrent_dropout = 0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

# encoder lstm layer2
encoder_lstm2 = LSTM(hidden_size, return_sequences=True,
                    return_state=True,
                    dropout = 0.4,
                    recurrent_dropout = 0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)
# encoder lstm layer3
encoder_lstm3 = LSTM(hidden_size, return_sequences=True,
                    return_state=True,
                    dropout = 0.4,
                    recurrent_dropout = 0.4)
encoder_outputs, state_h, state_c = encoder_lstm3(encoder_output2)

# decoder
decoder_inputs = Input(shape=(None,))

# decoder Embedding layer
dec_emb = Embedding(tarVoc, embedding_dim)(decoder_inputs)

# decoder lstm
decoder_lstm = LSTM(hidden_size, return_sequences=True, 
                    return_state=True,
                    dropout = 0.4,
                    recurrent_dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(dec_emb, 
                                     initial_state=[state_h,state_c])

# decoder output layer
decoder_softmax_layer = Dense(tarVoc, activation = 'softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_outputs)

# define model
model = Model([encoder_inputs, decoder_inputs],
              decoder_softmax_outputs)
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 128)      896000      input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 50, 256), (N 394240      embedding[0][0]                  
_____________________________________________________________________________________________

In [124]:
import urllib.request
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/thushv89/attention_keras/master/layers/attention.py",
    filename="attention.py")
from attention import Attentionlayer

ImportError: cannot import name 'Attentionlayer' from 'attention' (C:\Users\김경한\attention.py)

In [123]:
# 어텐션 층(어텐션 함수)
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# 어텐션의 결과와 디코더의 hidden state들을 연결
decoder_concat_input = Concatenate(axis = -1, name='concat_layer')([decoder_outputs, attn_out])

# 디코더의 출력층
decoder_softmax_layer = Dense(tarVoc, activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_concat_input)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 2)
history = model.fit([xTrain, yTrain[:,:-1]], yTrain.reshape(yTrain.shape[0], yTrain.shape[1], 1)[:,1:] \
                  ,epochs=50, callbacks=[es], batch_size = 256, validation_data=([xTest, yTest[:,:-1]], \
                  yTest.reshape(yTest.shape[0], yTest.shape[1], 1)[:,1:]))

plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()



src_index_to_word = srcToken.index_word # 원문 단어 집합에서 정수 -> 단어를 얻음
tar_word_to_index = tarToken.word_index # 요약 단어 집합에서 단어 -> 정수를 얻음
tar_index_to_word = tar_token.index_word # 요약 단어 집합에서 정수 -> 단어를 얻음

# 인코더 설계
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs, state_h, state_c])

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(hidden_size,))
decoder_state_input_c = Input(shape=(hidden_size,))

dec_emb2= dec_emb_layer(decoder_inputs) 
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# 어텐션 함수
decoder_hidden_state_input = Input(shape=(textMaxLen, hidden_size))
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# 디코더의 출력층
decoder_outputs2 = decoder_softmax_layer(decoder_inf_concat) 

# 최종 디코더 모델
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])


ImportError: cannot import name 'Attentionlayer' from 'attention' (C:\Users\김경한\attention.py)

In [ ]:

def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tarVoce))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition: #stop_condition이 True가 될 때까지 루프 반복
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > maxTarLen):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트 합니다.
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 상태를 업데이트 합니다.
        states_value = [h, c]

    return decoded_sentence

# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2text(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + src_index_to_word[i]+' '
    return temp

# 요약문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2summary(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostoken']) and i!=target_word_index['eostoken']):
            temp = temp + tar_index_to_word[i] + ' '
    return temp

for i in range(500, 1000):
    print("원문 : ",seq2text(xTest[i]))
    print("실제 요약문 :",seq2summary(yTest[i]))
    print("예측 요약문 :",decode_sequence(xTest[i].reshape(1, textMaxLen)))
    print("\n")